In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score,mean_squared_error

In [35]:
data_orig=pd.read_csv('../MA_PREDICTOR/data/ma_data_car.csv', parse_dates=['announcement_date'])

In [36]:
data = data_orig[(data_orig.car > -0.3) & (data_orig.car < 0.3)]

In [37]:
data.head()

,id,announcement_date,deal_value,acquiror_total_assets,acquiror_name,acquiror_ticker,target_name,target_nation,acquiror_nation,target_status,...,shares_acquired,consideration_offered,attitude,acquisition_technique,acquiror_financial_advisor,target_financial_advisor,purpose,acquiror_code,target_code,car
0,1631732020,2005-01-01,12.00,3160.41,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,100.0,Cash|Cash Only,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,50102030,50103030,0.006854
1,1653652020,2005-01-01,NaN,1680.57,Regis Corp,RGS,Scot Lewis Schools,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,54201030,63103010,-0.010266
2,1949421020,2005-01-01,NaN,3301.76,Google Inc,GOOG,PhatBits,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,57201030,57201020,0.007746
3,1652453040,2005-01-01,20.03,371.71,Graco Inc,GGG,Gusmer Europe SL,Spain,United States,Private,...,100.0,Cash|Cash Only,Friendly,Not Applicable,NaN,NaN,NaN,52102010,51101010,-0.011133
4,2721963020,2005-01-01,18220.52,NaN,Exxon Mobil Corp,XOM,Exxon Mobil Corp,United States,United States,Public,...,5.0,Cash|Cash Only,No Applicable,Open Market Purchase|Privately Negotiated Purc...,NaN,NaN,Other,50102030,50102030,-0.003971


In [38]:
# Prepare X and y
X = data[data.columns.difference(['car'])]
y = data['car']


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3, 
                                                    random_state = 6) # Holdout


In [39]:
y_test.shape

(5468,)

# Baseline with DummyRegressor

In [54]:
baseline_model_mean = DummyRegressor(strategy="mean") 

In [55]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='r2').mean()

-0.0002943093767784344

In [56]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.002128012109304252

In [57]:
list_strategy=['mean', "median", "constant"]

In [62]:
for strategy in list_strategy:
    constant = None
    if strategy =="constant":
        constant = 0
    baseline_model = DummyRegressor(strategy=strategy, constant=constant) 
    r2= cross_val_score(baseline_model, X_train, y_train, cv=5, scoring='r2').mean()
    mse= cross_val_score(baseline_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    print(f"{strategy} r2 : {r2}")
    print(f"{strategy} mse : {mse}")

mean r2 : -0.0002943093767784344
mean mse : -0.002128012109304252
median r2 : -0.004782366610396149
median mse : -0.0021376449643477043
constant r2 : -0.005824435294263441
constant mse : -0.002139867772719692


# First baseline: Mean

In [8]:
mean=y_train.mean(axis = 0, skipna = True)

In [9]:
mean

0.0034745868954703485

In [13]:
y_pred_mean= np.full(y_test.shape, fill_value=mean)

In [14]:
MSE_mean= mean_squared_error(y_test, y_pred_mean)

In [15]:
MSE_mean

0.0021552209448889984

# Second baseline : Median

In [16]:
median=y_train.median(axis = 0, skipna = True)

In [17]:
median

0.00033620874166315

In [18]:
y_pred_median= np.full(y_test.shape, fill_value=median)

In [19]:
MSE_median= mean_squared_error(y_test, y_pred_median)

In [20]:
MSE_median

0.002155361361108563

# Third Baseline: Constant

In [24]:
y_pred_zero= np.full(y_test.shape, fill_value=0)

In [25]:
MSE_zero= mean_squared_error(y_test, y_pred_zero)

In [26]:
MSE_zero

0.0021565445901306174